In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from dax.help_functions.get_dax_data import get_prepared_data
from evaluation.help_functions.prepare_data import next_working_days

### Quantile Regression only based on Time Series

In [3]:
daxdata = get_prepared_data()
daxdata
F

,Close,LogRetLag1,LogRetLag2,LogRetLag3,LogRetLag4,LogRetLag5
Date,,,,,,
2019-09-10 00:00:00+02:00,12268.709961,0.347914,0.629422,1.163586,2.006097,2.960150
2019-09-11 00:00:00+02:00,12359.070312,0.733812,1.081725,1.363234,1.897398,2.739909
2019-09-12 00:00:00+02:00,12410.250000,0.413251,1.147063,1.494977,1.776485,2.310649
2019-09-13 00:00:00+02:00,12468.530273,0.468515,0.881766,1.615578,1.963491,2.245000
2019-09-16 00:00:00+02:00,12380.309570,-0.710062,-0.241547,0.171704,0.905516,1.253429
...,...,...,...,...,...,...
2024-01-18 00:00:00+01:00,16567.349609,0.822211,-0.026133,-0.330653,-0.824790,0.122728
2024-01-19 00:00:00+01:00,16555.130859,-0.073779,0.748432,-0.099912,-0.404432,-0.898569
2024-01-22 00:00:00+01:00,16683.359375,0.771570,0.697791,1.520002,0.671658,0.367138


In [4]:
def get_quantile_regression_forecasts(daxdata=pd.DataFrame()):

    if daxdata.empty:
        daxdata = get_prepared_data()

    # create forecast frame
    column_names = [f'q{q}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]]
    dates = next_working_days(max(daxdata.index), 5)
    quantile_df = pd.DataFrame(index=dates, columns=column_names)

    R_t = pd.DataFrame(daxdata.iloc[-1:]['LogRetLag1'].copy().abs())
    R_t.insert(0, column='intercept', value=1)

    quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

    for h in range(1, 6):
        X = pd.DataFrame(daxdata.iloc[:-h][f'LogRetLag{h}'].copy().abs())
        X.insert(0, column='intercept', value=np.ones(shape=(len(daxdata)-h)))

        Y = daxdata[[f'LogRetLag{h}']].shift(-h).iloc[:-h].copy()
        Y.rename(columns={'LogRetLag1': "lr1dayahead"})

        model_qr_temp = sm.QuantReg(endog=Y, exog=X)

        for q in quantiles:
            model_quantile = model_qr_temp.fit(q=q)
            # Calculate forecasts for R_t using the fitted model for the current quantile
            forecast_temp = model_quantile.predict(R_t)
            quantile_df.loc[dates[h-1]][f'q{q}'] = forecast_temp[0]

    date_st = daxdata.index[-1].strftime('%Y-%m-%d')
    quantile_df.insert(0, 'forecast_date', date_st)
    quantile_df.insert(1, 'target', 'DAX')
    quantile_df.insert(
        2, "horizon", [str(i) + " day" for i in (1, 2, 5, 6, 7)])
    quantile_df.index.name = "date_time"

    return (quantile_df)

In [5]:
daxdata = get_prepared_data()

column_names = [f'q{q}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]]
dates = next_working_days(max(daxdata.index), 5)
quantile_df = pd.DataFrame(index=dates, columns=column_names)

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

for h in range(1, 6):

    name = f'LogRetLag{h}'
    X = pd.DataFrame(daxdata.iloc[:-h][name].copy())
    X.insert(0, column='intercept', value=np.ones(shape=(len(daxdata)-h)))

    Y = daxdata[[name]].shift(-h).iloc[:-h].copy()
    Y = Y.rename(columns={name: f"lr{h}daysahead"})
    model_qr_temp = sm.QuantReg(endog=Y, exog=X)
    
    R_t = pd.DataFrame(daxdata.iloc[-1:][name].copy())
    R_t.insert(0, column='intercept', value=1)

    for q in quantiles:
        model_quantile = model_qr_temp.fit(q=q)
        # Calculate forecasts for R_t using the fitted model for the current quantile
        forecast_temp = model_quantile.predict(R_t)
        quantile_df.loc[dates[h-1]][f'q{q}'] = forecast_temp[0]

date_st = daxdata.index[-1].strftime('%Y-%m-%d')
quantile_df.insert(0, 'forecast_date', date_st)
quantile_df.insert(1, 'target', 'DAX')
quantile_df.insert(
        2, "horizon", [str(i) + " day" for i in (1, 2, 5, 6, 7)])
quantile_df.index.name = "date_time"

quantile_df


,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-25,2024-01-24,DAX,1 day,-3.182953,-0.532448,-0.034406,0.649128,2.633715
2024-01-26,2024-01-24,DAX,2 day,-3.789289,-0.662812,0.045216,0.932469,3.481783
2024-01-29,2024-01-24,DAX,5 day,-4.11198,-0.810072,0.059267,0.893642,4.269547
2024-01-30,2024-01-24,DAX,6 day,-5.283738,-0.780883,0.203913,1.260747,4.931132
2024-01-31,2024-01-24,DAX,7 day,-5.513106,-0.68949,0.390359,1.377835,5.248098


In [6]:
from dax.help_functions.get_dax_data import get_prepared_data
daxdata = get_prepared_data()

In [20]:
import numpy as np
import pandas as pd

from dax.models.ARMA_GARCH.arma_garch import get_arma_garch11_forecasts, get_arma_garch_opt_pq_forecasts, get_arma_garch_opt_pq_lag_forecasts
from dax.models.baseline import get_dax_forecasts_baseline
from dax.models.QuantReg.quantile_regression import get_quantile_regression_forecasts
from dax.models.QuantReg.QuantReg import get_QuantReg_forecasts, get_QuantReg_forecasts_abs
from dax.models.QuantReg.QuantRegFeatures import get_QuantRegFeatures_forecasts
from dax.models.ARMA_NN.arma_nn import get_arma_nn_forecasts


def combine_models(daxdata=pd.DataFrame):

    QuantRegAbs_forecasts = get_QuantReg_forecasts_abs(daxdata)
    QuantRegFeat_Forecasts = get_QuantRegFeatures_forecasts(daxdata)
    ARMA_nn_Forecasts = get_arma_nn_forecasts(daxdata)
    Garch_pq = get_arma_garch_opt_pq_forecasts(daxdata)


    combined_model = QuantRegAbs_forecasts.copy()
    combined_model.loc[:, 'q0.025':'q0.975'] = np.nan

    combined_model.loc[:, 'q0.025':'q0.975'].iloc[0] = np.array(
        QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[0])*0.7 + np.array(QuantRegFeat_Forecasts.loc[:, 'q0.025':'q0.975'].iloc[0])*0.3
    combined_model.loc[:, 'q0.025':'q0.975'].iloc[1] = np.array(
        QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[1])*0.2 + np.array(QuantRegFeat_Forecasts.loc[:, 'q0.025':'q0.975'].iloc[1])*0.2
    combined_model.loc[:, 'q0.025':'q0.975'].iloc[2] = np.array(
        QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[2])*0.5 + np.array(QuantRegFeat_Forecasts.loc[:, 'q0.025':'q0.975'].iloc[2])*0.5
    combined_model.loc[:, 'q0.025':'q0.975'].iloc[3] = np.array(
        QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[3])*0.7 + np.array(Garch_pq.loc[:, 'q0.025':'q0.975'].iloc[3])*0.3
    combined_model.loc[:, 'q0.025':'q0.975'].iloc[4] = np.array(
        QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[4])*0.2 + np.array(ARMA_nn_Forecasts.loc[:, 'q0.025':'q0.975'].iloc[4])*0.8

    return combined_model

In [21]:
combined_model = combine_models(daxdata)
combined_model

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


update unemployment rate on: https://statistik.arbeitsagentur.de/DE/Navigation/Statistiken/Interaktive-Statistiken/Zeitreihen/Lange-Zeitreihen-Nav.html
1104
958


c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmo

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-1.94371,-0.331706,0.018696,0.492039,1.80371
2024-01-12,2024-01-10,DAX,2 day,-0.867044,-0.167619,0.02954,0.225364,0.815918
2024-01-15,2024-01-10,DAX,5 day,-2.80324,-0.565488,0.172514,0.768345,2.859062
2024-01-16,2024-01-10,DAX,6 day,-4.216667,-0.865925,0.146337,1.238233,3.500745
2024-01-17,2024-01-10,DAX,7 day,-5.760401,-1.975153,-0.214216,1.520983,4.961363


In [11]:
import numpy as np
QuantRegAbs_forecasts

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-2.299609,-0.398935,0.02436,0.563808,2.104597
2024-01-12,2024-01-10,DAX,2 day,-2.693969,-0.540463,0.12155,0.710513,2.679002
2024-01-15,2024-01-10,DAX,5 day,-4.033795,-0.703587,0.225472,1.097595,3.823407
2024-01-16,2024-01-10,DAX,6 day,-4.511936,-0.754442,0.209053,1.286311,3.48919
2024-01-17,2024-01-10,DAX,7 day,-5.289478,-0.869595,0.323588,1.388083,4.083629


In [17]:
combined_model.iloc[0] = np.array(
    QuantRegAbs_forecasts.iloc[0])*0.7 + np.array(QuantRegFeat_Forecasts.iloc[0])*0.3
combined_model

TypeError: can't multiply sequence by non-int of type 'float'

In [19]:
QuantRegAbs_forecasts
combined_model.loc[:,'q0.025':'q0.975'].iloc[0] = np.array(QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[0])*0.7 + np.array(QuantRegFeat_Forecasts.loc[:, 'q0.025':'q0.975'].iloc[0])*0.3
combined_model

C:\Users\Maria\AppData\Local\Temp\ipykernel_8240\1207446815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_model.loc[:,'q0.025':'q0.975'].iloc[0] = np.array(QuantRegAbs_forecasts.loc[:, 'q0.025':'q0.975'].iloc[0])*0.7 + np.array(QuantRegFeat_Forecasts.loc[:, 'q0.025':'q0.975'].iloc[0])*0.3


,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-1.94371,-0.331706,0.018696,0.492039,1.80371
2024-01-12,2024-01-10,DAX,2 day,NaN,NaN,NaN,NaN,NaN
2024-01-15,2024-01-10,DAX,5 day,NaN,NaN,NaN,NaN,NaN
2024-01-16,2024-01-10,DAX,6 day,NaN,NaN,NaN,NaN,NaN
2024-01-17,2024-01-10,DAX,7 day,NaN,NaN,NaN,NaN,NaN
